# Anàlisi Exploratòria de Dades (EDA)

Aquest notebook té com a objectiu analitzar i extreure insights dels datasets processats per crear la visualització final. 

S'analitzaran els següents datasets:
- `agri_country_year_df_processed.csv`: Dades agregades per país i any
- `agri_production_and_prices_df_processed.csv`: Dades desagregades per cultiu


In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# Visualització
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats

# Configuració de visualització
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuració de pandas per mostrar més informació
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.precision', 2)

In [ ]:
# Lectura dels datasets processats

data_dir = '../data/visualization_input_processed'

agri_country_year_df = pd.read_csv(
    os.path.join(data_dir, 'agri_country_year_df_processed.csv')
)

agri_production_prices_df = pd.read_csv(
    os.path.join(data_dir, 'agri_production_and_prices_df_processed.csv')
)

print("Dataset 1: agri_country_year_df_processed.csv")
print(f"Forma: {agri_country_year_df.shape}")
print(f"Columnes: {agri_country_year_df.columns.tolist()}")
display(agri_country_year_df.head())

print("Dataset 2: agri_production_and_prices_df_processed.csv")
print(f"Forma: {agri_production_prices_df.shape}")
print(f"Columnes: {agri_production_prices_df.columns.tolist()}")
display(agri_production_prices_df.head())


## Creació de sèrie temporal global: inputs (temperatura) i rendiment

Creem un DataFrame global que combina:
- Rendiment global anual (de global_climate_yield_ts)
- Temperatura mitjana anual per país (mitjana ponderada per població de cada país)


In [ ]:
# Creació del DataFrame global de sèries temporals: clima i rendiment

# PAS 1: Calcular temperatura mitjana anual global (ponderada per població)
# Filtrar files vàlides: temperatura i població han de ser vàlides i positives
valid_temp_data = agri_country_year_df[
    (agri_country_year_df['mean_annual_tas_deg_celsius'].notna()) &
    (agri_country_year_df['total_population_No'].notna()) &
    (agri_country_year_df['total_population_No'] > 0)
].copy()

print(f"Files vàlides per al càlcul de temperatura: {len(valid_temp_data):,} de {len(agri_country_year_df):,}")

# Calcular temperatura ponderada per població per any
# Global temperature = sum(temperature × population) / sum(population)
temp_global = (
    valid_temp_data
    .assign(
        weighted_temp_product=lambda x: x['mean_annual_tas_deg_celsius'] * x['total_population_No']
    )
    .groupby('Year')
    .agg({
        'weighted_temp_product': 'sum',
        'total_population_No': 'sum'
    })
    .assign(
        mean_annual_tas_deg_celsius=lambda x: x['weighted_temp_product'] / x['total_population_No']
    )
    .reset_index()
    [['Year', 'mean_annual_tas_deg_celsius']]
)

# Assegurar que Year sigui int
temp_global['Year'] = temp_global['Year'].astype(int)

print(f"Temperatura global (ponderada per població): {len(temp_global)} anys únics")
print(f"Rang d'anys (temperatura): {temp_global['Year'].min()} - {temp_global['Year'].max()}")


# PAS 2: Calcular rendiment global anual (ponderat per àrea)
# Filtrar files vàlides: ambdós yield_kg_ha i area_harvested_ha han de ser positius i no nuls
valid_yield_data = agri_production_prices_df[
    (agri_production_prices_df['yield_kg_ha'].notna()) &
    (agri_production_prices_df['area_harvested_ha'].notna()) &
    (agri_production_prices_df['yield_kg_ha'] > 0) &
    (agri_production_prices_df['area_harvested_ha'] > 0)
].copy()

print(f"\nFiles vàlides per al càlcul de rendiment: {len(valid_yield_data):,} de {len(agri_production_prices_df):,}")

# Calcular rendiment ponderat per any
# weighted_yield = sum(yield * area) / sum(area)
yield_global = (
    valid_yield_data
    .assign(
        weighted_product=lambda x: x['yield_kg_ha'] * x['area_harvested_ha']
    )
    .groupby('Year')
    .agg({
        'weighted_product': 'sum',
        'area_harvested_ha': 'sum'
    })
    .assign(
        yield_kg_ha=lambda x: x['weighted_product'] / x['area_harvested_ha']
    )
    .reset_index()
    [['Year', 'yield_kg_ha']]
)

# Assegurar que Year sigui int
yield_global['Year'] = yield_global['Year'].astype(int)

print(f"Rendiment global: {len(yield_global)} anys únics")
print(f"Rang d'anys (rendiment): {yield_global['Year'].min()} - {yield_global['Year'].max()}")


# PAS 3: Fusionar les dues sèries temporals
# Inner join per assegurar que només tenim anys amb dades de tots dos
global_climate_yield_ts = pd.merge(
    temp_global,
    yield_global,
    on='Year',
    how='inner'
)

# Ordenar per any (ascendent)
global_climate_yield_ts = global_climate_yield_ts.sort_values('Year').reset_index(drop=True)
# Assegurar que Year sigui int
global_climate_yield_ts['Year'] = global_climate_yield_ts['Year'].astype(int)


# Comprovació 1: No ha d'haver valors nuls en Year
assert global_climate_yield_ts['Year'].notna().all(), "ERROR: Hi ha valors nuls en Year!"

# Comprovació 2: Year ha de ser únic
assert global_climate_yield_ts['Year'].nunique() == len(global_climate_yield_ts), \
    "ERROR: Hi ha anys duplicats!"

# Comprovació 3: Els anys han d'estar ordenats
assert global_climate_yield_ts['Year'].is_monotonic_increasing, \
    "ERROR: Els anys no estan ordenats correctament!"

print("Comprovacions fetes: Totes passades.")


# PAS 5: Mostrar informació del DataFrame final
print(f"\nDataFrame final: global_climate_yield_ts")
print(f"Forma: {global_climate_yield_ts.shape}")
print(f"Columnes: {global_climate_yield_ts.columns.tolist()}")
print(f"\nNombre d'anys coberts: {len(global_climate_yield_ts)}")
print(f"Any mínim: {global_climate_yield_ts['Year'].min()}")
print(f"Any màxim: {global_climate_yield_ts['Year'].max()}")

print("\nPrimeres files:")
display(global_climate_yield_ts.head(10))

print("\nEstadístiques descriptives:")
display(global_climate_yield_ts.describe())


In [ ]:
# Guardar el DataFrame a disc
output_path = '../data/visualization_input_processed/global_climate_yield_ts.csv'
global_climate_yield_ts.to_csv(output_path, index=False)

print(f"DataFrame guardat correctament a: {output_path}")

In [ ]:
# Visualització de la sèrie temporal global: clima i rendiment
fig, ax1 = plt.subplots(figsize=(10, 5))

# Yield (left axis)
color_yield = "tab:blue"
ax1.plot(
    global_climate_yield_ts["Year"],
    global_climate_yield_ts["yield_kg_ha"],
    color=color_yield,
    linewidth=2
)
ax1.set_xlabel("Year")
ax1.set_ylabel("Global yield (kg/ha)", color=color_yield)
ax1.tick_params(axis="y", labelcolor=color_yield)

# Temperature (right axis)
ax2 = ax1.twinx()
color_temp = "tab:red"
ax2.plot(
    global_climate_yield_ts["Year"],
    global_climate_yield_ts["mean_annual_tas_deg_celsius"],
    color=color_temp,
    linewidth=2
)
ax2.set_ylabel("Mean annual temperature (°C)", color=color_temp)
ax2.tick_params(axis="y", labelcolor=color_temp)

plt.title("Global temperature and agricultural yield over time")
plt.tight_layout()
plt.show()


## Creació de sèrie temporal global: inputs (fertilizants) i rendiment

Creem un DataFrame global que combina:
- Rendiment global anual (de global_climate_yield_ts)
- Fertilizants per àrea de cultiu (mitjana ponderada per superfície de cultiu)


In [ ]:
# Creació del DataFrame global de sèries temporals: inputs (fertilizants) i rendiment

# Calcular fertilitzants globals ponderats per superfície de cultiu
# Filtrar files vàlides: fertilitzants i superfície de cultiu han de ser vàlids i positius
valid_fert_data = agri_country_year_df[
    (agri_country_year_df['fertilizers_per_area_of_cropland_kg_ha'].notna()) &
    (agri_country_year_df['cropland_ha'].notna()) &
    (agri_country_year_df['fertilizers_per_area_of_cropland_kg_ha'] >= 0) &
    (agri_country_year_df['cropland_ha'] > 0)
].copy()

print(f"Files vàlides per al càlcul de fertilitzants: {len(valid_fert_data):,} de {len(agri_country_year_df):,}")

# Calcular fertilitzants ponderats per superfície de cultiu per any
# fert_global = sum(fertilizers * cropland) / sum(cropland)
fert_global_ts = (
    valid_fert_data
    .assign(
        weighted_fert_product=lambda x: x['fertilizers_per_area_of_cropland_kg_ha'] * x['cropland_ha']
    )
    .groupby('Year')
    .agg({
        'weighted_fert_product': 'sum',
        'cropland_ha': 'sum'
    })
    .assign(
        fertilizers_per_area_of_cropland_kg_ha=lambda x: x['weighted_fert_product'] / x['cropland_ha']
    )
    .reset_index()
)

# Eliminar anys amb dades insuficients (sum(cropland_ha) == 0, encara que no hauria de passar després del filtre)
fert_global_ts = fert_global_ts[fert_global_ts['cropland_ha'] > 0].copy()

# Mantenir només Year i fertilizers_per_area_of_cropland_kg_ha
fert_global_ts = fert_global_ts[['Year', 'fertilizers_per_area_of_cropland_kg_ha']].copy()

# Assegurar que Year sigui int
fert_global_ts['Year'] = fert_global_ts['Year'].astype(int)

print(f"Fertilitzants globals (ponderats per superfície de cultiu): {len(fert_global_ts)} anys únics")
print(f"Rang d'anys (fertilitzants): {fert_global_ts['Year'].min()} - {fert_global_ts['Year'].max()}")

# Fusionar amb global_climate_yield_ts per obtenir yield_kg_ha
# Inner join per assegurar que només tenim anys amb dades de tots dos
global_inputs_yield_ts = pd.merge(
    global_climate_yield_ts[['Year', 'yield_kg_ha']],
    fert_global_ts,
    on='Year',
    how='inner'
)

# Mantenir només les columnes necessàries: Year, yield_kg_ha, fertilizers_per_area_of_cropland_kg_ha
global_inputs_yield_ts = global_inputs_yield_ts[['Year', 'yield_kg_ha', 'fertilizers_per_area_of_cropland_kg_ha']].copy()

# Ordenar per any (ascendent)
global_inputs_yield_ts = global_inputs_yield_ts.sort_values('Year').reset_index(drop=True)

# Assegurar que Year sigui int
global_inputs_yield_ts['Year'] = global_inputs_yield_ts['Year'].astype(int)

# Comprovacions
assert global_inputs_yield_ts['Year'].notna().all(), "Error: Hi ha valors nuls en Year"
assert global_inputs_yield_ts['Year'].nunique() == len(global_inputs_yield_ts), "Error: Hi ha anys duplicats"
assert global_inputs_yield_ts['Year'].is_monotonic_increasing, "Error: Els anys no estan ordenats correctament"
assert global_inputs_yield_ts['yield_kg_ha'].notna().all(), "Error: Hi ha valors nuls en yield_kg_ha"
assert global_inputs_yield_ts['fertilizers_per_area_of_cropland_kg_ha'].notna().all(), "Error: Hi ha valors nuls en fertilizers_per_area_of_cropland_kg_ha"

print("\nComprovacions: totes passades")

# Mostrar informació del DataFrame final
print(f"\nDataFrame final: global_inputs_yield_ts")
print(f"Forma: {global_inputs_yield_ts.shape}")
print(f"Columnes: {global_inputs_yield_ts.columns.tolist()}")
print(f"\nNombre d'anys coberts: {len(global_inputs_yield_ts)}")
print(f"Any mínim: {global_inputs_yield_ts['Year'].min()}")
print(f"Any màxim: {global_inputs_yield_ts['Year'].max()}")

print("\nPrimeres files:")
display(global_inputs_yield_ts.head())

print("\nEstadístiques descriptives:")
display(global_inputs_yield_ts.describe())


In [ ]:
# Guardar el DataFrame a disc
output_path = '../data/visualization_input_processed/global_inputs_fertilizers_yield_ts.csv'
global_inputs_yield_ts.to_csv(output_path, index=False)

print(f"DataFrame guardat correctament a: {output_path}")

In [ ]:
# Visualització exploratòria: comparació de rendiment i fertilitzants en un sol gràfic

fig, ax1 = plt.subplots(figsize=(12, 6))

color1 = 'tab:blue'
color2 = 'tab:green'

# Gràfic de rendiment (eix y primari)
ax1.plot(
    global_inputs_yield_ts['Year'], 
    global_inputs_yield_ts['yield_kg_ha'], 
    color=color1, 
    linewidth=2, 
    label='Rendiment global (kg/ha)'
)
ax1.set_xlabel('Any', fontsize=11)
ax1.set_ylabel('Rendiment global (kg/ha)', color=color1, fontsize=11)
ax1.tick_params(axis='y', labelcolor=color1)
ax1.grid(True, alpha=0.3)

# Crear segon eix y per a fertilitzants
ax2 = ax1.twinx()
ax2.plot(
    global_inputs_yield_ts['Year'], 
    global_inputs_yield_ts['fertilizers_per_area_of_cropland_kg_ha'], 
    color=color2, 
    linewidth=2, 
    label='Fertilitzants per àrea de cultiu (kg/ha)'
)
ax2.set_ylabel('Fertilitzants per àrea de cultiu (kg/ha)', color=color2, fontsize=11)
ax2.tick_params(axis='y', labelcolor=color2)

# Llegenda combinada per ambdues sèries
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left', fontsize=10)

plt.title('Comparació del rendiment agrícola global i l\'ús de fertilitzants al llarg del temps', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()


## Creació de sèrie temporal global: emissions i eficiència

Creem un DataFrame global que analitza el cost ambiental de la productivitat agrícola:
- Rendiment global anual (de global_inputs_yield_ts)
- Emissions per hectàrea (intensitat d'emissions agrícoles globals)


In [ ]:
# Creació del DataFrame global de sèries temporals: emissions i eficiència

# Calcular emissions globals per hectàrea
# Filtrar files vàlides: emissions i superfície de cultiu han de ser vàlids i positius
valid_emissions_data = agri_country_year_df[
    (agri_country_year_df['emissions_total_agriculture_CO2eq_AR5_kt'].notna()) &
    (agri_country_year_df['cropland_ha'].notna()) &
    (agri_country_year_df['emissions_total_agriculture_CO2eq_AR5_kt'] >= 0) &
    (agri_country_year_df['cropland_ha'] > 0)
].copy()

print(f"Files vàlides per al càlcul d'emissions: {len(valid_emissions_data):,} de {len(agri_country_year_df):,}")

# Agregar per any: sumar emissions totals i superfície de cultiu
emissions_global_ts = (
    valid_emissions_data
    .groupby('Year')
    .agg({
        'emissions_total_agriculture_CO2eq_AR5_kt': 'sum',
        'cropland_ha': 'sum'
    })
    .reset_index()
)

# Calcular emissions per hectàrea: sum(emissions) / sum(cropland)
emissions_global_ts['emissions_per_ha_CO2eq_kt'] = (
    emissions_global_ts['emissions_total_agriculture_CO2eq_AR5_kt'] / 
    emissions_global_ts['cropland_ha']
)

# Eliminar anys amb dades insuficients (sum(cropland_ha) == 0)
emissions_global_ts = emissions_global_ts[emissions_global_ts['cropland_ha'] > 0].copy()

# Mantenir només Year i emissions_per_ha_CO2eq_kt
emissions_global_ts = emissions_global_ts[['Year', 'emissions_per_ha_CO2eq_kt']].copy()

# Assegurar que Year sigui int
emissions_global_ts['Year'] = emissions_global_ts['Year'].astype(int)

print(f"Emissions globals per hectàrea: {len(emissions_global_ts)} anys únics")
print(f"Rang d'anys (emissions): {emissions_global_ts['Year'].min()} - {emissions_global_ts['Year'].max()}")

# Fusionar amb global_inputs_yield_ts per obtenir yield_kg_ha
# Inner join per assegurar que només tenim anys amb dades de tots dos
global_emissions_efficiency_ts = pd.merge(
    global_inputs_yield_ts[['Year', 'yield_kg_ha']],
    emissions_global_ts,
    on='Year',
    how='inner'
)

# Mantenir només les columnes necessàries: Year, yield_kg_ha, emissions_per_ha_CO2eq_kt
global_emissions_efficiency_ts = global_emissions_efficiency_ts[
    ['Year', 'yield_kg_ha', 'emissions_per_ha_CO2eq_kt']
].copy()

# Ordenar per any (ascendent)
global_emissions_efficiency_ts = global_emissions_efficiency_ts.sort_values('Year').reset_index(drop=True)

# Assegurar que Year sigui int
global_emissions_efficiency_ts['Year'] = global_emissions_efficiency_ts['Year'].astype(int)

# Comprovacions
assert global_emissions_efficiency_ts['Year'].notna().all(), "Error: Hi ha valors nuls en Year"
assert global_emissions_efficiency_ts['Year'].nunique() == len(global_emissions_efficiency_ts), "Error: Hi ha anys duplicats"
assert global_emissions_efficiency_ts['Year'].is_monotonic_increasing, "Error: Els anys no estan ordenats correctament"
assert global_emissions_efficiency_ts['yield_kg_ha'].notna().all(), "Error: Hi ha valors nuls en yield_kg_ha"
assert global_emissions_efficiency_ts['emissions_per_ha_CO2eq_kt'].notna().all(), "Error: Hi ha valors nuls en emissions_per_ha_CO2eq_kt"

print("\nComprovacions: totes passades")

# Mostrar informació del DataFrame final
print(f"\nDataFrame final: global_emissions_efficiency_ts")
print(f"Forma: {global_emissions_efficiency_ts.shape}")
print(f"Columnes: {global_emissions_efficiency_ts.columns.tolist()}")
print(f"\nNombre d'anys coberts: {len(global_emissions_efficiency_ts)}")
print(f"Any mínim: {global_emissions_efficiency_ts['Year'].min()}")
print(f"Any màxim: {global_emissions_efficiency_ts['Year'].max()}")

print("\nPrimeres files:")
display(global_emissions_efficiency_ts.head())

print("\nEstadístiques descriptives:")
display(global_emissions_efficiency_ts.describe())


In [ ]:
# Veien que tots els valors de emisions son de l'ordre de 10^-3, pasem de kt a t
global_emissions_efficiency_ts['emissions_per_ha_CO2eq_t'] = (
    global_emissions_efficiency_ts['emissions_per_ha_CO2eq_kt'] * 1000
)
global_emissions_efficiency_ts = global_emissions_efficiency_ts.drop(
    columns=['emissions_per_ha_CO2eq_kt']
)


In [ ]:
# Guardar el DataFrame a disc
output_path = '../data/visualization_input_processed/global_emissions_efficiency_ts.csv'
global_emissions_efficiency_ts.to_csv(output_path, index=False)

print(f"DataFrame guardat correctament a: {output_path}")

In [ ]:
# Visualització exploratòria: rendiment vs emissions per hectàrea

fig, ax1 = plt.subplots(figsize=(10, 6))

color1 = 'tab:blue'
color2 = 'tab:orange'

# Gràfic de rendiment (eix y primari)
ax1.plot(
    global_emissions_efficiency_ts['Year'], 
    global_emissions_efficiency_ts['yield_kg_ha'], 
    color=color1, 
    linewidth=2, 
    label='Rendiment global (kg/ha)'
)
ax1.set_xlabel('Any', fontsize=11)
ax1.set_ylabel('Rendiment global (kg/ha)', color=color1, fontsize=11)
ax1.tick_params(axis='y', labelcolor=color1)

# Crear segon eix y per a emissions
ax2 = ax1.twinx()
ax2.plot(
    global_emissions_efficiency_ts['Year'], 
    global_emissions_efficiency_ts['emissions_per_ha_CO2eq_t'], 
    color=color2, 
    linewidth=2, 
    label='Emissions per hectàrea (t CO₂eq/ha)'
)
ax2.set_ylabel('Emissions per hectàrea (t CO₂eq/ha)', color=color2, fontsize=11)
ax2.tick_params(axis='y', labelcolor=color2)

# Llegenda combinada per ambdues sèries
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left', fontsize=10)

plt.title('Rendiment agrícola global i intensitat d\'emissions al llarg del temps', fontsize=12)
plt.tight_layout()
plt.show()


## Índex de Vulnerabilitat Climàtica Agrícola

Creem un índex de vulnerabilitat que combina:
- Estrès climàtic (temperatura i precipitació)
- Dependència econòmica (pes de l'agricultura al PIB)
- Exposició social (població rural)


In [ ]:
# Creació de l'índex de vulnerabilitat climàtica agrícola per país

# Definir períodes
baseline_start = 1961
baseline_end = 1990
recent_start = 2014
recent_end = 2023

print(f"Període de referència (baseline): {baseline_start}-{baseline_end}")
print(f"Període recent: {recent_start}-{recent_end}")

# Filtrar dades per període de referència (precipitació baseline)
baseline_data = agri_country_year_df[
    (agri_country_year_df['Year'] >= baseline_start) & 
    (agri_country_year_df['Year'] <= baseline_end)
].copy()

# Filtrar dades per període recent
recent_data = agri_country_year_df[
    (agri_country_year_df['Year'] >= recent_start) & 
    (agri_country_year_df['Year'] <= recent_end)
].copy()

print(f"\nFiles al període baseline: {len(baseline_data):,}")
print(f"Files al període recent: {len(recent_data):,}")

# Calcular mitjanes per país al període recent
recent_avg = (
    recent_data
    .groupby(['ISO3', 'Country'])
    .agg({
        'surface_temperature_change_celsius': 'mean',
        'prec_mm_per_year': 'mean',
        'agriculture_share_gdp_percent': 'mean',
        'rural_population_%': 'mean'
    })
    .reset_index()
    .rename(columns={
        'surface_temperature_change_celsius': 'temp_recent_avg',
        'prec_mm_per_year': 'recent_prec',
        'agriculture_share_gdp_percent': 'agriculture_share_gdp_recent_avg',
        'rural_population_%': 'rural_population_recent_avg'
    })
)

# Calcular precipitació baseline per país
baseline_prec = (
    baseline_data
    .groupby(['ISO3', 'Country'])
    .agg({
        'prec_mm_per_year': 'mean'
    })
    .reset_index()
    .rename(columns={'prec_mm_per_year': 'baseline_prec'})
)

# Fusionar dades recent i baseline
country_vulnerability_df = pd.merge(
    recent_avg,
    baseline_prec,
    on=['ISO3', 'Country'],
    how='inner'
)

# Calcular estrès de precipitació (desviació absoluta respecte al baseline)
country_vulnerability_df['prec_stress'] = abs(
    country_vulnerability_df['recent_prec'] - country_vulnerability_df['baseline_prec']
)

# Afegir informació del període
country_vulnerability_df['recent_period_start'] = recent_start
country_vulnerability_df['recent_period_end'] = recent_end

print(f"\nPaïsos amb dades completes: {len(country_vulnerability_df)}")

# Eliminar països amb valors nuls en qualsevol component necessari
required_cols = [
    'temp_recent_avg', 'baseline_prec', 'recent_prec', 'prec_stress',
    'agriculture_share_gdp_recent_avg', 'rural_population_recent_avg'
]

country_vulnerability_df = country_vulnerability_df.dropna(subset=required_cols)

print(f"Països després d'eliminar valors nuls: {len(country_vulnerability_df)}")

country_vulnerability_df['temp_z'] = stats.zscore(country_vulnerability_df['temp_recent_avg'], nan_policy='omit')
country_vulnerability_df['prec_z'] = stats.zscore(country_vulnerability_df['prec_stress'], nan_policy='omit')
country_vulnerability_df['agdep_z'] = stats.zscore(country_vulnerability_df['agriculture_share_gdp_recent_avg'], nan_policy='omit')
country_vulnerability_df['rural_z'] = stats.zscore(country_vulnerability_df['rural_population_recent_avg'], nan_policy='omit')

# Eliminar qualsevol fila amb z-scores nuls (hauria de ser rar després de dropna)
country_vulnerability_df = country_vulnerability_df.dropna(subset=['temp_z', 'prec_z', 'agdep_z', 'rural_z'])

print(f"Països finals amb tots els components: {len(country_vulnerability_df)}")

# Calcular índex de vulnerabilitat (suma ponderada)
country_vulnerability_df['vulnerability_index'] = (
    0.35 * country_vulnerability_df['temp_z'] +
    0.25 * country_vulnerability_df['prec_z'] +
    0.25 * country_vulnerability_df['agdep_z'] +
    0.15 * country_vulnerability_df['rural_z']
)

# Escalar a 0-100
vuln_min = country_vulnerability_df['vulnerability_index'].min()
vuln_max = country_vulnerability_df['vulnerability_index'].max()
country_vulnerability_df['vulnerability_0_100'] = (
    (country_vulnerability_df['vulnerability_index'] - vuln_min) / 
    (vuln_max - vuln_min) * 100
)

# Ordenar per vulnerabilitat descendent
country_vulnerability_df = country_vulnerability_df.sort_values(
    'vulnerability_0_100', 
    ascending=False
).reset_index(drop=True)

# Reordenar columnes segons especificació
column_order = [
    'ISO3', 'Country',
    'recent_period_start', 'recent_period_end',
    'temp_recent_avg', 'baseline_prec', 'recent_prec', 'prec_stress',
    'agriculture_share_gdp_recent_avg', 'rural_population_recent_avg',
    'temp_z', 'prec_z', 'agdep_z', 'rural_z',
    'vulnerability_index', 'vulnerability_0_100'
]

country_vulnerability_df = country_vulnerability_df[column_order]

# Comprovacions
assert country_vulnerability_df['vulnerability_0_100'].min() >= 0, "Error: vulnerabilitat mínima < 0"
assert country_vulnerability_df['vulnerability_0_100'].max() <= 100, "Error: vulnerabilitat màxima > 100"
assert country_vulnerability_df['ISO3'].nunique() == len(country_vulnerability_df), "Error: ISO3 duplicats"

print("\nComprovacions: totes passades")

# Mostrar informació
print(f"\nDataFrame final: country_vulnerability_df")
print(f"Nombre de països: {len(country_vulnerability_df)}")
print(f"\nPrimeres files:")
display(country_vulnerability_df.head(10))

print(f"\nTop 10 països més vulnerables:")
top_10 = country_vulnerability_df[['Country', 'vulnerability_0_100']].head(10)
for idx, row in top_10.iterrows():
    print(f"  {row['Country']}: {row['vulnerability_0_100']:.2f}")


In [ ]:
# Guardar el DataFrame a disc
output_path = '../data/visualization_input_processed/country_vulnerability_df.csv'
country_vulnerability_df.to_csv(output_path, index=False)

print(f"DataFrame guardat correctament a: {output_path}")
print(f"Fitxer creat amb {len(country_vulnerability_df)} files i {len(country_vulnerability_df.columns)} columnes")


In [ ]:
# Visualització: mapa coroplètic de vulnerabilitat climàtica agrícola

fig = px.choropleth(
    country_vulnerability_df,
    locations="ISO3",
    color="vulnerability_0_100",
    hover_name="Country",
    hover_data=[
        'temp_recent_avg',
        'prec_stress',
        'agriculture_share_gdp_recent_avg',
        'rural_population_recent_avg',
        'vulnerability_0_100'
    ],
    title="Agricultural Climate Vulnerability Index (Recent vs Baseline)",
    color_continuous_scale='RdYlBu_r',  # Red-Yellow-Blue reversed (red = high vulnerability)
    labels={'vulnerability_0_100': 'Vulnerability Index (0-100)'}
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=600
)

fig.show()

print(f"\nNombre de països inclosos al mapa: {len(country_vulnerability_df)}")
print(f"\nTop 10 països més vulnerables:")
display(country_vulnerability_df[['Country', 'vulnerability_0_100']].head(10))
print(f"\nTop 10 països menys vulnerables:")
display(country_vulnerability_df[['Country', 'vulnerability_0_100']].tail(10))


## Índex d'Eficiència Agrícola Sostenible

Creem un índex d'eficiencia que combina:
- Alt rendiment (kg/ha)
- Baix ús de fertilitzants (kg/ha)
- Baixes emissions (kt CO₂eq/ha)


In [ ]:
# Creació de l'índex d'eficiencia agrícola sostenible per país

# Utilitzem el mateix període recent (2014-2023 o últims 10 anys disponibles)

print(f"Període recent utilitzat: {recent_start}-{recent_end}")

# A) Calcular rendiment per país (ponderat per àrea de cultiu)
print("\nCalculant rendiment per país...")

# Filtrar dades de producció del període recent
prod_recent = agri_production_prices_df[
    (agri_production_prices_df['Year'] >= recent_start) & 
    (agri_production_prices_df['Year'] <= recent_end)
].copy()

# Filtrar files vàlides
prod_recent = prod_recent[
    (prod_recent['area_harvested_ha'].notna()) &
    (prod_recent['production_kt'].notna()) &
    (prod_recent['area_harvested_ha'] > 0) &
    (prod_recent['production_kt'] > 0)
].copy()

# Agregar duplicats a nivell ISO3-Year-Commodity
prod_agg = (
    prod_recent
    .groupby(['ISO3', 'Country', 'Year', 'Commodity'])
    .agg({
        'area_harvested_ha': 'sum',
        'production_kt': 'sum'
    })
    .reset_index()
)

# Recalcular yield després de l'agregació
prod_agg['yield_kg_ha'] = (prod_agg['production_kt'] * 1_000_000) / prod_agg['area_harvested_ha']

# Filtrar files amb yield vàlid i positiu
prod_agg = prod_agg[
    (prod_agg['yield_kg_ha'].notna()) &
    (prod_agg['yield_kg_ha'] > 0)
].copy()

# Calcular rendiment ponderat per àrea per país-any
yield_country_year = (
    prod_agg
    .assign(
        weighted_yield_product=lambda x: x['yield_kg_ha'] * x['area_harvested_ha']
    )
    .groupby(['ISO3', 'Country', 'Year'])
    .agg({
        'weighted_yield_product': 'sum',
        'area_harvested_ha': 'sum'
    })
    .assign(
        yield_country_year=lambda x: x['weighted_yield_product'] / x['area_harvested_ha']
    )
    .reset_index()
    [['ISO3', 'Country', 'Year', 'yield_country_year']]
)

# Mitjana per país al període recent
yield_recent_avg = (
    yield_country_year
    .groupby(['ISO3', 'Country'])
    .agg({
        'yield_country_year': 'mean'
    })
    .reset_index()
    .rename(columns={'yield_country_year': 'yield_recent_avg_kg_ha'})
)

print(f"Països amb dades de rendiment: {len(yield_recent_avg)}")

# B) Calcular fertilitzants per país (mitjana recent)
print("\nCalculant fertilitzants per país...")

fert_recent = agri_country_year_df[
    (agri_country_year_df['Year'] >= recent_start) & 
    (agri_country_year_df['Year'] <= recent_end) &
    (agri_country_year_df['fertilizers_per_area_of_cropland_kg_ha'].notna()) &
    (agri_country_year_df['fertilizers_per_area_of_cropland_kg_ha'] >= 0)
].copy()

fertilizers_recent_avg = (
    fert_recent
    .groupby(['ISO3', 'Country'])
    .agg({
        'fertilizers_per_area_of_cropland_kg_ha': 'mean'
    })
    .reset_index()
    .rename(columns={'fertilizers_per_area_of_cropland_kg_ha': 'fertilizers_recent_avg_kg_ha'})
)

print(f"Països amb dades de fertilitzants: {len(fertilizers_recent_avg)}")

# C) Calcular emissions per hectàrea per país (mitjana recent)
print("\nCalculant emissions per país...")

emis_recent = agri_country_year_df[
    (agri_country_year_df['Year'] >= recent_start) & 
    (agri_country_year_df['Year'] <= recent_end) &
    (agri_country_year_df['emissions_total_agriculture_CO2eq_AR5_kt'].notna()) &
    (agri_country_year_df['cropland_ha'].notna()) &
    (agri_country_year_df['emissions_total_agriculture_CO2eq_AR5_kt'] >= 0) &
    (agri_country_year_df['cropland_ha'] > 0)
].copy()

# Calcular emissions per hectàrea per país-any
emis_per_ha = emis_recent.copy()
emis_per_ha['emissions_per_ha'] = (
    emis_per_ha['emissions_total_agriculture_CO2eq_AR5_kt'] / 
    emis_per_ha['cropland_ha']
)

emissions_recent_avg = (
    emis_per_ha
    .groupby(['ISO3', 'Country'])
    .agg({
        'emissions_per_ha': 'mean'
    })
    .reset_index()
    .rename(columns={'emissions_per_ha': 'emissions_recent_avg_kt_per_ha'})
)

print(f"Països amb dades d'emissions: {len(emissions_recent_avg)}")

# D) Opcional: agriculture_share_gdp_percent
print("\nCalculant pes de l'agricultura al PIB...")

gdp_recent = agri_country_year_df[
    (agri_country_year_df['Year'] >= recent_start) & 
    (agri_country_year_df['Year'] <= recent_end) &
    (agri_country_year_df['agriculture_share_gdp_percent'].notna())
].copy()

agriculture_share_gdp_recent_avg = (
    gdp_recent
    .groupby(['ISO3', 'Country'])
    .agg({
        'agriculture_share_gdp_percent': 'mean'
    })
    .reset_index()
    .rename(columns={'agriculture_share_gdp_percent': 'agriculture_share_gdp_recent_avg'})
)

print(f"Països amb dades de PIB agrícola: {len(agriculture_share_gdp_recent_avg)}")

# Fusionar totes les mètriques
country_efficiency_df = yield_recent_avg.copy()

country_efficiency_df = pd.merge(
    country_efficiency_df,
    fertilizers_recent_avg,
    on=['ISO3', 'Country'],
    how='inner'
)

country_efficiency_df = pd.merge(
    country_efficiency_df,
    emissions_recent_avg,
    on=['ISO3', 'Country'],
    how='inner'
)

# Afegir PIB agrícola (opcional, left join)
country_efficiency_df = pd.merge(
    country_efficiency_df,
    agriculture_share_gdp_recent_avg,
    on=['ISO3', 'Country'],
    how='left'
)

# Afegir informació del període
country_efficiency_df['recent_period_start'] = recent_start
country_efficiency_df['recent_period_end'] = recent_end

# Eliminar països amb valors nuls en les mètriques principals
required_cols = ['yield_recent_avg_kg_ha', 'fertilizers_recent_avg_kg_ha', 'emissions_recent_avg_t_per_ha']
country_efficiency_df = country_efficiency_df.dropna(subset=required_cols)

# Multiplicar per 1000 i canviar de kt per ha a t per ha
country_efficiency_df['emissions_recent_avg_t_per_ha'] = (
    country_efficiency_df['emissions_recent_avg_kt_per_ha'] * 1000
)
# Eliminar la columna antiga i canviar el nom per consistència a la resta del notebook
country_efficiency_df = country_efficiency_df.drop(columns=['emissions_recent_avg_kt_per_ha'])

# Assegurar que no hi ha valors nuls en ISO3/Country
country_efficiency_df = country_efficiency_df[
    country_efficiency_df['ISO3'].notna() & 
    country_efficiency_df['Country'].notna()
].copy()

print(f"\nPaïsos finals amb totes les mètriques: {len(country_efficiency_df)}")

country_efficiency_df['yield_z'] = stats.zscore(country_efficiency_df['yield_recent_avg_kg_ha'], nan_policy='omit')
country_efficiency_df['fert_z'] = stats.zscore(country_efficiency_df['fertilizers_recent_avg_kg_ha'], nan_policy='omit')
country_efficiency_df['emis_z'] = stats.zscore(country_efficiency_df['emissions_recent_avg_t_per_ha'], nan_policy='omit')

# Eliminar files amb z-scores nuls
country_efficiency_df = country_efficiency_df.dropna(subset=['yield_z', 'fert_z', 'emis_z'])

# Calcular índex d'eficiencia
# Alt rendiment és bo (+), alt fertilitzant és dolent (-), altes emissions són dolentes (-)
country_efficiency_df['efficiency_index'] = (
    0.45 * country_efficiency_df['yield_z'] - 
    0.30 * country_efficiency_df['fert_z'] - 
    0.25 * country_efficiency_df['emis_z']
)

# Escalar a 0-100
eff_min = country_efficiency_df['efficiency_index'].min()
eff_max = country_efficiency_df['efficiency_index'].max()
country_efficiency_df['efficiency_0_100'] = (
    (country_efficiency_df['efficiency_index'] - eff_min) / 
    (eff_max - eff_min) * 100
)

# Ordenar per eficiencia descendent
country_efficiency_df = country_efficiency_df.sort_values(
    'efficiency_0_100', 
    ascending=False
).reset_index(drop=True)

# Reordenar columnes segons especificació
column_order = [
    'ISO3', 'Country',
    'recent_period_start', 'recent_period_end',
    'yield_recent_avg_kg_ha',
    'fertilizers_recent_avg_kg_ha',
    'emissions_recent_avg_t_per_ha',
    'agriculture_share_gdp_recent_avg',
    'yield_z', 'fert_z', 'emis_z',
    'efficiency_index', 'efficiency_0_100'
]

# Mantenir només les columnes que existeixen
column_order = [col for col in column_order if col in country_efficiency_df.columns]
country_efficiency_df = country_efficiency_df[column_order]

# Comprovacions
assert country_efficiency_df['efficiency_0_100'].min() >= 0, "Error: eficiencia mínima < 0"
assert country_efficiency_df['efficiency_0_100'].max() <= 100, "Error: eficiencia màxima > 100"
assert country_efficiency_df['ISO3'].nunique() == len(country_efficiency_df), "Error: ISO3 duplicats"

print("\nComprovacions: totes passades")

# Mostrar informació
print(f"\nDataFrame final: country_efficiency_df")
print(f"Nombre de països: {len(country_efficiency_df)}")
print(f"\nPrimeres files (més eficients):")
display(country_efficiency_df.head(10))

print(f"\nÚltimes files (menys eficients):")
display(country_efficiency_df.tail(10))


Arribats a aquest punt veiem que el valor de Faroe Islands és un outlier i està introduint molt de soroll a l'índex. Provoracà que tots els índex siguin alts. Per tant l'eliminem i tornem a calcular els índexos. De fet, l'elimiem dels dataframes originals per que no sigui un outlier a cap de les mètriques.

In [ ]:
# Mostrar top 10 més eficients i menys eficients amb les 3 mètriques

print("Top 10 països més eficients:")
top_10_efficient = country_efficiency_df[['Country', 'efficiency_0_100', 'yield_recent_avg_kg_ha', 
                                          'fertilizers_recent_avg_kg_ha', 'emissions_recent_avg_t_per_ha']].head(10)
for idx, row in top_10_efficient.iterrows():
    print(f"  {row['Country']}: Eficiencia={row['efficiency_0_100']:.2f}, "
          f"Rendiment={row['yield_recent_avg_kg_ha']:.1f} kg/ha, "
          f"Fertilitzants={row['fertilizers_recent_avg_kg_ha']:.1f} kg/ha, "
          f"Emissions={row['emissions_recent_avg_t_per_ha']:.4f} kt/ha")

print("\nTop 10 països menys eficients:")
bottom_10_efficient = country_efficiency_df[['Country', 'efficiency_0_100', 'yield_recent_avg_kg_ha', 
                                              'fertilizers_recent_avg_kg_ha', 'emissions_recent_avg_t_per_ha']].tail(10)
for idx, row in bottom_10_efficient.iterrows():
    print(f"  {row['Country']}: Eficiencia={row['efficiency_0_100']:.2f}, "
          f"Rendiment={row['yield_recent_avg_kg_ha']:.1f} kg/ha, "
          f"Fertilitzants={row['fertilizers_recent_avg_kg_ha']:.1f} kg/ha, "
          f"Emissions={row['emissions_recent_avg_t_per_ha']:.4f} kt/ha")


In [ ]:
# Guardar el DataFrame a disc
output_path = '../data/visualization_input_processed/country_efficiency_df.csv'
country_efficiency_df.to_csv(output_path, index=False)

print(f"DataFrame guardat correctament a: {output_path}")
print(f"Fitxer creat amb {len(country_efficiency_df)} files i {len(country_efficiency_df.columns)} columnes")


In [ ]:
# Visualització: mapa coroplètic d'eficiencia agrícola

fig = px.choropleth(
    country_efficiency_df,
    locations="ISO3",
    color="efficiency_0_100",
    hover_name="Country",
    hover_data=[
        'yield_recent_avg_kg_ha',
        'fertilizers_recent_avg_kg_ha',
        'emissions_recent_avg_t_per_ha',
        'efficiency_0_100'
    ],
    title="Agricultural Efficiency Index (High yield, low inputs, low emissions)",
    color_continuous_scale='RdYlGn',  # Red-Yellow-Green (green = high efficiency)
    labels={'efficiency_0_100': 'Efficiency Index (0-100)'}
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=600
)

fig.show()

print(f"\nNombre de països inclosos al mapa: {len(country_efficiency_df)}")


In [ ]:
# Visualització exploratòria: scatter plot rendiment vs fertilitzants

fig, ax = plt.subplots(figsize=(10, 7))

scatter = ax.scatter(
    country_efficiency_df['fertilizers_recent_avg_kg_ha'],
    country_efficiency_df['yield_recent_avg_kg_ha'],
    c=country_efficiency_df['efficiency_0_100'],
    cmap='RdYlGn',
    s=50,
    alpha=0.6,
    edgecolors='black',
    linewidth=0.5
)

ax.set_xlabel('Fertilitzants per hectàrea (kg/ha)', fontsize=11)
ax.set_ylabel('Rendiment per hectàrea (kg/ha)', fontsize=11)
ax.set_title('Rendiment vs Fertilitzants (colorat per índex d\'eficiencia)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Afegir colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Índex d\'Eficiencia (0-100)', fontsize=10)

plt.tight_layout()
plt.show()


## Estructura de Gènere en l'Ocupació Agrícola

Anàlisi descriptiva de l'estructura de gènere en l'ocupació agrícola a nivell de país i regional.

In [ ]:
# Creació del dataset d'estructura de gènere a nivell de país

# Determinar període recent (2015-2024 o últims 10 anys disponibles)
target_start = 2015
target_end = 2024

# Verificar quins anys estan disponibles
available_years = sorted(agri_country_year_df['Year'].unique())
recent_years = [y for y in available_years if y >= target_start and y <= target_end]

# Si no hi ha suficients anys, agafar els últims 10 disponibles
if len(recent_years) < 10:
    recent_years = sorted(available_years)[-10:]

recent_start = min(recent_years)
recent_end = max(recent_years)

print(f"Període recent utilitzat: {recent_start}-{recent_end} ({len(recent_years)} anys)")

# Filtrar dades del període recent
gender_recent = agri_country_year_df[
    (agri_country_year_df['Year'] >= recent_start) & 
    (agri_country_year_df['Year'] <= recent_end)
].copy()

print(f"\nFiles al període recent: {len(gender_recent):,}")

# Filtrar valors vàlids (>= 0) per a les variables d'interès
gender_recent = gender_recent[
    (gender_recent['female_share_of_employment_in_agriculture'].notna() |
     gender_recent['total_share_of_employment_in_agriculture'].notna())
].copy()

# Aplicar filtres de valors >= 0 per a cada variable
if 'female_share_of_employment_in_agriculture' in gender_recent.columns:
    gender_recent = gender_recent[
        (gender_recent['female_share_of_employment_in_agriculture'].isna()) |
        (gender_recent['female_share_of_employment_in_agriculture'] >= 0)
    ].copy()

if 'total_share_of_employment_in_agriculture' in gender_recent.columns:
    gender_recent = gender_recent[
        (gender_recent['total_share_of_employment_in_agriculture'].isna()) |
        (gender_recent['total_share_of_employment_in_agriculture'] >= 0)
    ].copy()

# Calcular mitjanes per país al període recent
country_gender_structure_df = (
    gender_recent
    .groupby(['ISO3', 'Country'])
    .agg({
        'female_share_of_employment_in_agriculture': 'mean',
        'total_share_of_employment_in_agriculture': 'mean'
    })
    .reset_index()
)

print(f"Països amb dades de gènere i ocupació: {len(country_gender_structure_df)}")

# Eliminar països amb valors nuls en qualsevol de les dues mètriques (requisits obligatoris)
country_gender_structure_df = country_gender_structure_df[
    country_gender_structure_df['female_share_of_employment_in_agriculture'].notna() &
    country_gender_structure_df['total_share_of_employment_in_agriculture'].notna()
].copy()

# Assegurar que no hi ha valors nuls en ISO3/Country
country_gender_structure_df = country_gender_structure_df[
    country_gender_structure_df['ISO3'].notna() & 
    country_gender_structure_df['Country'].notna()
].copy()

# Assegurar una fila per ISO3 (eliminar duplicats si n'hi ha)
country_gender_structure_df = country_gender_structure_df.drop_duplicates(subset=['ISO3']).copy()

# Reordenar columnes segons especificació
column_order = [
    'ISO3', 'Country',
    'female_share_of_employment_in_agriculture',
    'total_share_of_employment_in_agriculture'
]

country_gender_structure_df = country_gender_structure_df[column_order]

# Comprovacions de qualitat
assert country_gender_structure_df['ISO3'].nunique() == len(country_gender_structure_df), "Error: ISO3 duplicats"
assert country_gender_structure_df['female_share_of_employment_in_agriculture'].notna().all(), \
    "Error: Hi ha valors nuls en female_share_of_employment_in_agriculture"
assert country_gender_structure_df['total_share_of_employment_in_agriculture'].notna().all(), \
    "Error: Hi ha valors nuls en total_share_of_employment_in_agriculture"

print("\nComprovacions de qualitat: totes passades")

# Mostrar informació
print(f"\nDataFrame final: country_gender_structure_df")
print(f"Nombre de països: {len(country_gender_structure_df)}")
print(f"\nPrimeres files:")
display(country_gender_structure_df.head(10))

print(f"\nEstadístiques descriptives:")
display(country_gender_structure_df.describe())


In [ ]:
# Mostrar top 10 països per participació de dones
print("Top 10 països per participació de dones en ocupació agrícola:")
top_10_female = country_gender_structure_df.nlargest(
    10, 
    'female_share_of_employment_in_agriculture'
)[['Country', 'female_share_of_employment_in_agriculture']]

for idx, row in top_10_female.iterrows():
    print(f"  {row['Country']}: {row['female_share_of_employment_in_agriculture']:.2%}")


In [ ]:
# Guardar el DataFrame a disc
output_path = '../data/visualization_input_processed/country_gender_structure_df.csv'
country_gender_structure_df.to_csv(output_path, index=False)

print(f"DataFrame guardat correctament a: {output_path}")
print(f"Fitxer creat amb {len(country_gender_structure_df)} files i {len(country_gender_structure_df.columns)} columnes")


In [ ]:
# Visualització: mapa coroplètic de participació de dones en agricultura

fig = px.choropleth(
    country_gender_structure_df,
    locations="ISO3",
    color="female_share_of_employment_in_agriculture",
    hover_name="Country",
    hover_data=[
        'female_share_of_employment_in_agriculture',
        'total_share_of_employment_in_agriculture'
    ],
    title="Female Share of Employment in Agriculture (Recent Average)",
    labels={'female_share_of_employment_in_agriculture': 'Female Share (%)'}
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=600
)

fig.show()

print(f"\nNombre de països inclosos al mapa: {len(country_gender_structure_df)}")


Les dades semblan estar contaminades o no ser del tot correctes. No és realista pensar que a tota Europa el percentatge de representació femenina sigui per sota del 5%. Per tant, no introduirem aquesta variable a al nostra visualització i ho afagirem a l'apartat de dificultats trobades.

## Composició d'Emissions Agrícoles Globals

Anàlisi de la composició de les emissions agrícoles globals per component, adequat per a gràfics d'àrea apilats a Flourish.


In [ ]:
# Creació de datasets de composició d'emissions agrícoles globals

# Definir components d'emissions i les seves columnes corresponents
emission_components = {
    'soils': 'emissions_soils_CO2eq_AR5_kt',
    'rice': 'emissions_rice_cultivation_agricultural_CO2eq_AR5_kt',
    'synthetic_fertilizers': 'emissions_synthetic_fertilizers_CO2eq_AR5_kt',
    'crop_residues': 'emissions_crop_residues_CO2eq_AR5_kt',
    'livestock': 'emissions_total_livestock_CO2eq_AR5_kt'
}

print("Components d'emissions:")
for comp, col in emission_components.items():
    print(f"  {comp}: {col}")

# Seleccionar dades necessàries
emissions_data = agri_country_year_df[['Year'] + list(emission_components.values())].copy()

print(f"\nFiles inicials: {len(emissions_data):,}")
print(f"Rang d'anys: {emissions_data['Year'].min()} - {emissions_data['Year'].max()}")

# Neteja: reemplaçar valors negatius amb NaN
for col in emission_components.values():
    if col in emissions_data.columns:
        negative_count = (emissions_data[col] < 0).sum()
        if negative_count > 0:
            print(f"  Reemplaçant {negative_count} valors negatius amb NaN a {col}")
            emissions_data.loc[emissions_data[col] < 0, col] = np.nan

# Agregar a nivell global per any
# Per cada any, sumar tots els països per cada component
global_emissions = emissions_data.groupby('Year').agg({
    col: 'sum' for col in emission_components.values()
}).reset_index()

print(f"\nAnys únics després de l'agregació: {len(global_emissions)}")

# Renombrar columnes amb noms més curts
column_rename = {v: f'{k}_kt' for k, v in emission_components.items()}
global_emissions = global_emissions.rename(columns=column_rename)

# Eliminar anys on tots els components són NaN
# (simplificat: eliminar anys on qualsevol component és completament nul)
before_drop = len(global_emissions)
component_cols = [f'{k}_kt' for k in emission_components.keys()]
global_emissions = global_emissions[
    global_emissions[component_cols].notna().any(axis=1)
].copy()

if len(global_emissions) < before_drop:
    print(f"  Eliminats {before_drop - len(global_emissions)} anys amb dades insuficients")

# Calcular total per any
global_emissions['total_kt'] = global_emissions[component_cols].sum(axis=1, skipna=True)

# Eliminar anys on el total és 0 o NaN
global_emissions = global_emissions[
    (global_emissions['total_kt'].notna()) & 
    (global_emissions['total_kt'] > 0)
].copy()

# Assegurar que Year sigui int
global_emissions['Year'] = global_emissions['Year'].astype(int)
global_emissions = global_emissions.sort_values('Year').reset_index(drop=True)

print(f"\nAnys finals amb dades vàlides: {len(global_emissions)}")
print(f"Rang d'anys final: {global_emissions['Year'].min()} - {global_emissions['Year'].max()}")

# Crear DataFrame wide absolut
global_emissions_composition_wide = global_emissions.copy()

print(f"\nDataFrame wide absolut:")
print(f"Forma: {global_emissions_composition_wide.shape}")
print(f"Columnes: {global_emissions_composition_wide.columns.tolist()}")
display(global_emissions_composition_wide.head(10))


In [ ]:
# Crear DataFrame wide percentatge
global_emissions_composition_pct_wide = global_emissions[['Year']].copy()

# Calcular percentatges per cada component
for comp in emission_components.keys():
    col_kt = f'{comp}_kt'
    col_pct = f'{comp}_pct'
    if col_kt in global_emissions.columns:
        global_emissions_composition_pct_wide[col_pct] = (
            global_emissions[col_kt] / global_emissions['total_kt'] * 100
        )

print(f"DataFrame wide percentatge:")
print(f"Forma: {global_emissions_composition_pct_wide.shape}")
print(f"Columnes: {global_emissions_composition_pct_wide.columns.tolist()}")
display(global_emissions_composition_pct_wide.head(10))


In [ ]:
# Crear DataFrame long format
global_emissions_composition_long = (
    global_emissions_composition_wide
    .melt(
        id_vars='Year',
        var_name='origin',
        value_name='value'
    )
    .pivot(index='origin', columns='Year', values='value')
    .reset_index()
)

global_emissions_composition_long.head()

In [ ]:
# Guardar tots els DataFrames a disc
output_dir = '../data/visualization_input_processed'

# Wide absolut
output_path_wide = os.path.join(output_dir, 'global_emissions_composition_wide.csv')
global_emissions_composition_wide.to_csv(output_path_wide, index=False)
print(f"Guardat: {output_path_wide}")

# Wide percentatge
output_path_pct = os.path.join(output_dir, 'global_emissions_composition_pct_wide.csv')
global_emissions_composition_pct_wide.to_csv(output_path_pct, index=False)
print(f"Guardat: {output_path_pct}")

# Long format
output_path_long = os.path.join(output_dir, 'global_emissions_composition_long.csv')
global_emissions_composition_long.to_csv(output_path_long, index=False)
print(f"Guardat: {output_path_long}")

print(f"\nTots els fitxers guardats correctament")
print(f"Rang d'anys: {global_emissions_composition_wide['Year'].min()} - {global_emissions_composition_wide['Year'].max()}")


In [ ]:
# Visualització: gràfic d'àrea apilat amb valors absoluts

fig, ax = plt.subplots(figsize=(12, 7))

# Preparar dades per a l'àrea apilada
years = global_emissions_composition_wide['Year'].values
components = ['soils_kt', 'rice_kt', 'synthetic_fertilizers_kt', 'crop_residues_kt', 'livestock_kt']
component_labels = ['Soils', 'Rice Cultivation', 'Synthetic Fertilizers', 'Crop Residues', 'Livestock']
colors = ['#8B4513', '#FFD700', '#32CD32', '#FF6347', '#4169E1']

# Crear gràfic d'àrea apilat
ax.stackplot(
    years,
    *[global_emissions_composition_wide[comp].fillna(0).values for comp in components],
    labels=component_labels,
    colors=colors,
    alpha=0.7
)

ax.set_xlabel('Any', fontsize=11)
ax.set_ylabel('Emissions (kt CO₂eq)', fontsize=11)
ax.set_title('Composició d\'Emissions Agrícoles Globals (Valors Absoluts)', fontsize=12, fontweight='bold')
ax.legend(loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Visualització: gràfic d'àrea apilat amb valors percentuals

fig, ax = plt.subplots(figsize=(12, 7))

# Preparar dades per a l'àrea apilada percentual
years = global_emissions_composition_pct_wide['Year'].values
components_pct = ['soils_pct', 'rice_pct', 'synthetic_fertilizers_pct', 'crop_residues_pct', 'livestock_pct']
component_labels = ['Soils', 'Rice Cultivation', 'Synthetic Fertilizers', 'Crop Residues', 'Livestock']
colors = ['#8B4513', '#FFD700', '#32CD32', '#FF6347', '#4169E1']

# Crear gràfic d'àrea apilat percentual
ax.stackplot(
    years,
    *[global_emissions_composition_pct_wide[comp].fillna(0).values for comp in components_pct],
    labels=component_labels,
    colors=colors,
    alpha=0.7
)

ax.set_xlabel('Any', fontsize=11)
ax.set_ylabel('Percentatge (%)', fontsize=11)
ax.set_title('Composició d\'Emissions Agrícoles Globals (Percentatges)', fontsize=12, fontweight='bold')
ax.set_ylim(0, 100)
ax.legend(loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Perfil Agrícola Regional

Anàlisi del perfil estructural de l'agricultura per regió, mostrant tres mètriques percentuals com a perfil comparatiu (no com a composició que suma 100%).


In [ ]:
# Creació de datasets de perfil agrícola regional

# Mapping ISO3 a Regió (reutilitzat del mapping anterior)
iso3_to_region = {
    # Àfrica
    'DZA': 'Africa', 'AGO': 'Africa', 'BEN': 'Africa', 'BWA': 'Africa', 'BFA': 'Africa',
    'BDI': 'Africa', 'CPV': 'Africa', 'CMR': 'Africa', 'CAF': 'Africa', 'TCD': 'Africa',
    'COM': 'Africa', 'COG': 'Africa', 'CIV': 'Africa', 'COD': 'Africa', 'DJI': 'Africa',
    'EGY': 'Africa', 'GNQ': 'Africa', 'ERI': 'Africa', 'SWZ': 'Africa', 'ETH': 'Africa',
    'GAB': 'Africa', 'GMB': 'Africa', 'GHA': 'Africa', 'GIN': 'Africa', 'GNB': 'Africa',
    'KEN': 'Africa', 'LSO': 'Africa', 'LBR': 'Africa', 'LBY': 'Africa', 'MDG': 'Africa',
    'MWI': 'Africa', 'MLI': 'Africa', 'MRT': 'Africa', 'MUS': 'Africa', 'MAR': 'Africa',
    'MOZ': 'Africa', 'NAM': 'Africa', 'NER': 'Africa', 'NGA': 'Africa', 'RWA': 'Africa',
    'STP': 'Africa', 'SEN': 'Africa', 'SYC': 'Africa', 'SLE': 'Africa', 'SOM': 'Africa',
    'ZAF': 'Africa', 'SSD': 'Africa', 'SDN': 'Africa', 'TZA': 'Africa', 'TGO': 'Africa',
    'TUN': 'Africa', 'UGA': 'Africa', 'ZMB': 'Africa', 'ZWE': 'Africa',

    # Àsia
    'AFG': 'Asia', 'ARM': 'Asia', 'AZE': 'Asia', 'BHR': 'Asia', 'BGD': 'Asia',
    'BTN': 'Asia', 'BRN': 'Asia', 'KHM': 'Asia', 'CHN': 'Asia', 'CYP': 'Asia',
    'GEO': 'Asia', 'IND': 'Asia', 'IDN': 'Asia', 'IRN': 'Asia', 'IRQ': 'Asia',
    'ISR': 'Asia', 'JPN': 'Asia', 'JOR': 'Asia', 'KAZ': 'Asia', 'KWT': 'Asia',
    'KGZ': 'Asia', 'LAO': 'Asia', 'LBN': 'Asia', 'MYS': 'Asia', 'MDV': 'Asia',
    'MNG': 'Asia', 'MMR': 'Asia', 'NPL': 'Asia', 'PRK': 'Asia', 'OMN': 'Asia',
    'PAK': 'Asia', 'PSE': 'Asia', 'PHL': 'Asia', 'QAT': 'Asia', 'SAU': 'Asia',
    'SGP': 'Asia', 'LKA': 'Asia', 'SYR': 'Asia', 'TWN': 'Asia', 'TJK': 'Asia',
    'THA': 'Asia', 'TLS': 'Asia', 'TUR': 'Asia', 'TKM': 'Asia', 'ARE': 'Asia',
    'UZB': 'Asia', 'VNM': 'Asia', 'YEM': 'Asia',
    'KOR': 'Asia', 'HKG': 'Asia',

    # Europa
    'ALB': 'Europe', 'AND': 'Europe', 'AUT': 'Europe', 'BLR': 'Europe', 'BEL': 'Europe',
    'BIH': 'Europe', 'BGR': 'Europe', 'HRV': 'Europe', 'CZE': 'Europe', 'DNK': 'Europe',
    'EST': 'Europe', 'FIN': 'Europe', 'FRA': 'Europe', 'DEU': 'Europe', 'GRC': 'Europe',
    'HUN': 'Europe', 'ISL': 'Europe', 'IRL': 'Europe', 'ITA': 'Europe', 'LVA': 'Europe',
    'LIE': 'Europe', 'LTU': 'Europe', 'LUX': 'Europe', 'MLT': 'Europe', 'MDA': 'Europe',
    'MCO': 'Europe', 'MNE': 'Europe', 'NLD': 'Europe', 'MKD': 'Europe', 'NOR': 'Europe',
    'POL': 'Europe', 'PRT': 'Europe', 'ROU': 'Europe', 'RUS': 'Europe', 'SMR': 'Europe',
    'SRB': 'Europe', 'SVK': 'Europe', 'SVN': 'Europe', 'ESP': 'Europe', 'SWE': 'Europe',
    'CHE': 'Europe', 'UKR': 'Europe', 'GBR': 'Europe', 'VAT': 'Europe',
    'GLP': 'Europe', 'GRL': 'Europe', 'MTQ': 'Europe', 'REU': 'Europe',

    # Amèriques
    'ARG': 'Americas', 'BHS': 'Americas', 'BRB': 'Americas', 'BLZ': 'Americas', 'BOL': 'Americas',
    'BRA': 'Americas', 'CAN': 'Americas', 'CHL': 'Americas', 'COL': 'Americas', 'CRI': 'Americas',
    'CUB': 'Americas', 'DMA': 'Americas', 'DOM': 'Americas', 'ECU': 'Americas', 'SLV': 'Americas',
    'GRD': 'Americas', 'GTM': 'Americas', 'GUY': 'Americas', 'HTI': 'Americas', 'HND': 'Americas',
    'JAM': 'Americas', 'MEX': 'Americas', 'NIC': 'Americas', 'PAN': 'Americas', 'PRY': 'Americas',
    'PER': 'Americas', 'KNA': 'Americas', 'LCA': 'Americas', 'VCT': 'Americas', 'SUR': 'Americas',
    'TTO': 'Americas', 'USA': 'Americas', 'URY': 'Americas', 'VEN': 'Americas',
    'ABW': 'Americas', 'ATG': 'Americas', 'BMU': 'Americas', 'PRI': 'Americas', 'GUF': 'Americas',

    # Oceania
    'AUS': 'Oceania', 'FJI': 'Oceania', 'KIR': 'Oceania', 'MHL': 'Oceania', 'FSM': 'Oceania',
    'NRU': 'Oceania', 'NZL': 'Oceania', 'PLW': 'Oceania', 'PNG': 'Oceania', 'WSM': 'Oceania',
    'SLB': 'Oceania', 'TON': 'Oceania', 'TUV': 'Oceania', 'VUT': 'Oceania',
    'COK': 'Oceania', 'NCL': 'Oceania', 'NIU': 'Oceania', 'PYF': 'Oceania',
}

print(f"Mapping ISO3 a Regió creat amb {len(iso3_to_region)} codis")

# Determinar període recent (2014-2023)
recent_start = 2014
recent_end = 2023

print(f"\nPeríode recent utilitzat: {recent_start}-{recent_end}")

# Filtrar dades del període recent
profile_data = agri_country_year_df[
    (agri_country_year_df['Year'] >= recent_start) & 
    (agri_country_year_df['Year'] <= recent_end)
].copy()

print(f"Files al període recent: {len(profile_data):,}")

# Filtrar valors vàlids (>= 0) per a les variables d'interès
for col in ['rural_population_%', 'total_share_of_employment_in_agriculture', 'agriculture_share_gdp_percent']:
    if col in profile_data.columns:
        profile_data = profile_data[
            (profile_data[col].isna()) | (profile_data[col] >= 0)
        ].copy()

# Variables per fer els pesos
weight_cols = ['total_population_No', 'GDP_usd']
# Elimina pesos invàlids (<=0) on existeixin (si són NaN es mantenen, es filtren al moment de ponderar)
for wcol in weight_cols:
    if wcol in profile_data.columns:
        profile_data = profile_data[(profile_data[wcol].isna()) | (profile_data[wcol] > 0)].copy()


# Calcular mitjanes per país en el període recent
# (valors simples per país; la ponderació només s'aplica al sumar per regió/global)
country_profile = (
    profile_data
    .groupby(['ISO3', 'Country'], as_index=False)
    .agg({
        'rural_population_%': 'mean',
        'total_share_of_employment_in_agriculture': 'mean',
        'agriculture_share_gdp_percent': 'mean',
        'total_population_No': 'mean',   # pes representat del país en el període
        'GDP_usd': 'mean'                # pes representat del país en el període
    })
)

print(f"Països amb dades de perfil: {len(country_profile)}")

# Afegir columna Region
country_profile['Region'] = country_profile['ISO3'].map(iso3_to_region)

# Identificar països sense mapping
missing_region = country_profile[country_profile['Region'].isna()]
if len(missing_region) > 0:
    print(f"\nPaïsos sense mapping de regió ({len(missing_region)}):")
    print(missing_region[['ISO3', 'Country']].to_string())
    print("\nEliminant països sense regió assignada...")

# Eliminar països sense regió assignada
country_profile = country_profile[country_profile['Region'].notna()].copy()

print(f"Països amb regió assignada: {len(country_profile)}")


# Funcions auxiliars de mitjana ponderada
def weighted_mean(values: pd.Series, weights: pd.Series) -> float:
    mask = values.notna() & weights.notna() & (weights > 0)
    if mask.sum() == 0:
        return np.nan
    v = values[mask].astype(float)
    w = weights[mask].astype(float)
    return float((v * w).sum() / w.sum())

# Agregació per regió (ponderada)
# - rural_population_% ponderat per total_population_No
# - employment_in_agri ponderat per total_population_No
# - agriculture_share_gdp_percent ponderat per GDP_usd
region_rows = []
for region, g in country_profile.groupby('Region'):
    row = {
        'Region': region,
        'rural_population_pct': weighted_mean(g['rural_population_%'], g['total_population_No']),
        'employment_in_agri_pct': weighted_mean(g['total_share_of_employment_in_agriculture'], g['total_population_No']),
        'agriculture_share_gdp_pct': weighted_mean(g['agriculture_share_gdp_percent'], g['GDP_usd']),
    }
    region_rows.append(row)

region_profile = pd.DataFrame(region_rows)

# Afegir el valor Global
global_row = {
    'Region': 'Global',
    'rural_population_pct': weighted_mean(country_profile['rural_population_%'], country_profile['total_population_No']),
    'employment_in_agri_pct': weighted_mean(country_profile['total_share_of_employment_in_agriculture'], country_profile['total_population_No']),
    'agriculture_share_gdp_pct': weighted_mean(country_profile['agriculture_share_gdp_percent'], country_profile['GDP_usd']),
}

region_profile = pd.concat([region_profile, pd.DataFrame([global_row])], ignore_index=True)

# Convertir a porcentaje si les columnes estaven en [0,1]
region_profile['rural_population_pct'] = region_profile['rural_population_pct'] * 100
region_profile['employment_in_agri_pct'] = region_profile['employment_in_agri_pct'] * 100

print(f"\nRegions amb dades (incloent Global): {len(region_profile)}")
display(region_profile)

In [ ]:
# Crear DataFrame wide format
region_agri_profile_wide = region_profile.copy()

print(f"DataFrame wide format:")
print(f"Forma: {region_agri_profile_wide.shape}")
print(f"Columnes: {region_agri_profile_wide.columns.tolist()}")
display(region_agri_profile_wide)


In [ ]:
# Crear DataFrame long format
long_data = []

for _, row in region_profile.iterrows():
    region = row['Region']
    
    # rural_population_pct
    long_data.append({
        'Region': region,
        'metric': 'rural_population_pct',
        'value': row['rural_population_pct']
    })
    
    # employment_in_agri_pct
    long_data.append({
        'Region': region,
        'metric': 'employment_in_agri_pct',
        'value': row['employment_in_agri_pct']
    })
    
    # agriculture_share_gdp_pct
    long_data.append({
        'Region': region,
        'metric': 'agriculture_share_gdp_pct',
        'value': row['agriculture_share_gdp_pct']
    })

region_agri_profile_long = pd.DataFrame(long_data)

print(f"DataFrame long format:")
print(f"Forma: {region_agri_profile_long.shape}")
print(f"Columnes: {region_agri_profile_long.columns.tolist()}")
display(region_agri_profile_long.head(15))


In [ ]:
# Wide format
output_path_wide = os.path.join(output_dir, 'region_agri_profile_wide.csv')
region_agri_profile_wide.to_csv(output_path_wide, index=False)
print(f"Guardat: {output_path_wide}")

# Long format
output_path_long = os.path.join(output_dir, 'region_agri_profile_long.csv')
region_agri_profile_long.to_csv(output_path_long, index=False)
print(f"Guardat: {output_path_long}")

print(f"\nTots els fitxers guardats correctament")


In [ ]:
# Mostrar taula ordenada per agriculture_share_gdp_pct descendent
region_sorted = region_agri_profile_wide.sort_values(
    'agriculture_share_gdp_pct',
    ascending=False
)

print("Perfil agrícola regional ordenat per pes de l'agricultura al PIB (descendent):")
display(region_sorted)


In [ ]:
# Visualització: gràfic de barres agrupades

fig, ax = plt.subplots(figsize=(12, 7))

# Preparar dades
regions = region_agri_profile_wide['Region'].values
x = np.arange(len(regions))
width = 0.25

# Valors per a cada mètrica
rural_vals = region_agri_profile_wide['rural_population_pct'].values
employment_vals = region_agri_profile_wide['employment_in_agri_pct'].values
gdp_vals = region_agri_profile_wide['agriculture_share_gdp_pct'].values

# Crear barres
bars1 = ax.bar(x - width, rural_vals, width, label='Població Rural (%)', color='#4CAF50', alpha=0.8)
bars2 = ax.bar(x, employment_vals, width, label='Ocupació en Agricultura (%)', color='#2196F3', alpha=0.8)
bars3 = ax.bar(x + width, gdp_vals, width, label='Pes Agricultura al PIB (%)', color='#FF9800', alpha=0.8)

ax.set_xlabel('Regió', fontsize=11)
ax.set_ylabel('Percentatge (%)', fontsize=11)
ax.set_title('Perfil Agrícola Regional: Tres Mètriques Estructurals', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(regions, rotation=45, ha='right')
ax.legend(loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# 1) CSV per Grouped bars
#    Region | metric | value_pct
df_viz1_grouped_bars = region_agri_profile_wide.melt(
    id_vars=["Region"],
    value_vars=["rural_population_pct", "employment_in_agri_pct", "agriculture_share_gdp_pct"],
    var_name="metric",
    value_name="value_pct"
)

metric_map = {
    "rural_population_pct": "Rural population (%)",
    "employment_in_agri_pct": "Employment in agriculture (%)",
    "agriculture_share_gdp_pct": "Agriculture share of GDP (%)",
}
df_viz1_grouped_bars["metric"] = df_viz1_grouped_bars["metric"].map(metric_map)

output_path = os.path.join(
    output_dir,
    "flourish_viz1_grouped_bars.csv"
)
df_viz1_grouped_bars.to_csv(output_path, index=False)

# 2) CSV per Scatter plot
#    Region | employment_in_agri_pct (X) | agriculture_share_gdp_pct (Y)
#    (+ rural_population_pct para tamaño, + gap y ratio para tooltips)
df_viz2_scatter = region_agri_profile_wide[
    ["Region", "employment_in_agri_pct", "agriculture_share_gdp_pct", "rural_population_pct"]
].copy()

df_viz2_scatter["gap_employment_minus_gdp_pp"] = (
    df_viz2_scatter["employment_in_agri_pct"] - df_viz2_scatter["agriculture_share_gdp_pct"]
)

df_viz2_scatter["employment_to_gdp_ratio"] = (
    df_viz2_scatter["employment_in_agri_pct"] / df_viz2_scatter["agriculture_share_gdp_pct"]
).replace([float("inf"), -float("inf")], pd.NA)

output_path = os.path.join(
    output_dir,
    "flourish_viz2_scatter.csv"
)
df_viz2_scatter.to_csv(output_path, index=False)

# 3) CSV per Slope chart
#    Region | Employment in agriculture (%) | Agriculture share of GDP (%)
df_viz3_slope = region_agri_profile_wide[
    ["Region", "employment_in_agri_pct", "agriculture_share_gdp_pct"]
].rename(columns={
    "employment_in_agri_pct": "Employment in agriculture (%)",
    "agriculture_share_gdp_pct": "Agriculture share of GDP (%)"
})
output_path = os.path.join(
    output_dir,
    "flourish_viz3_slope.csv"
)
df_viz3_slope.to_csv(output_path, index=False)

# 4) CSV per Ratio bar + filter (Global vs Continents)
#    Region | employment_to_gdp_ratio | gap | group
df_viz4_ratio = region_agri_profile_wide[
    ["Region", "employment_in_agri_pct", "agriculture_share_gdp_pct", "rural_population_pct"]
].copy()

df_viz4_ratio["employment_to_gdp_ratio"] = (
    df_viz4_ratio["employment_in_agri_pct"] / df_viz4_ratio["agriculture_share_gdp_pct"]
)

df_viz4_ratio["gap_employment_minus_gdp_pp"] = (
    df_viz4_ratio["employment_in_agri_pct"] - df_viz4_ratio["agriculture_share_gdp_pct"]
)

df_viz4_ratio["group"] = df_viz4_ratio["Region"].apply(lambda r: "Global" if r == "Global" else "Continent")

output_path = os.path.join(
    output_dir,
    "flourish_viz4_ratio_filter.csv"
)
df_viz4_ratio.to_csv(output_path, index=False)

# 5) CSV per Stacked bars (composición a 100%)
#    Region | Agriculture employment (%) | Rural non-agriculture (%) | Urban population (%)
df_viz5_stacked = region_agri_profile_wide[
    ["Region", "rural_population_pct", "employment_in_agri_pct"]
].copy()

df_viz5_stacked["Agriculture employment (%)"] = df_viz5_stacked["employment_in_agri_pct"]
df_viz5_stacked["Rural non-agriculture (%)"] = (
    df_viz5_stacked["rural_population_pct"] - df_viz5_stacked["employment_in_agri_pct"]
).clip(lower=0)

df_viz5_stacked["Urban population (%)"] = (100 - df_viz5_stacked["rural_population_pct"]).clip(lower=0)

df_viz5_stacked = df_viz5_stacked[
    ["Region", "Agriculture employment (%)", "Rural non-agriculture (%)", "Urban population (%)"]
]
output_path = os.path.join(
    output_dir,
    "flourish_viz5_stacked.csv"
)
df_viz5_stacked.to_csv(output_path, index=False)

print("CSVs guardados en:", output_dir)

In [ ]:
# Definició de dades generals amb la regió

# 1) Selecciona columnes
cols = [
    "Country",
    "Year",
    "ISO3",
    "total_population_No",
    "rural_population_%",
    "agriculture_share_gdp_percent",
    "total_share_of_employment_in_agriculture",
    "GDP_usd",
]

df = agri_country_year_df.loc[:, cols].copy()

# 2) Afegeix la Regió
df["Region"] = df["ISO3"].map(iso3_to_region)

# 3) Converteix proporcions a percentatges
pct_cols = [
    "rural_population_%",
    "total_share_of_employment_in_agriculture",
]
df[pct_cols] = df[pct_cols].mul(100)

# 1) Redondear los valores de porcentaje a enteros (0-100)
cols_to_round = pct_cols + ['agriculture_share_gdp_percent']
df[cols_to_round] = df[cols_to_round].round(0).clip(lower=0, upper=100).astype(int)

# 2) Convertir la columna Year en tipo fecha (sólo año)
df["Year"] = pd.to_datetime(df["Year"], format="%Y").dt.year


# 4) Desa el fitxer
output_path = os.path.join(
    output_dir,
    "agri_country_year_df_with_region.csv"
)
df.to_csv(output_path, index=False)
